<a href="https://colab.research.google.com/github/cl812958191/BigDataAnalytics/blob/master/%E2%80%9CColab_SocialMedia_Example_Spark_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Spark installation on Colab
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark
!rm -rf spark-3.0.1-bin-hadoop3.2.tgz

In [ ]:
# Set JAVA_HOME and SPARK_HOME
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.0.1-bin-hadoop3.2"

import findspark
findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME


import sys
import requests
from operator import add

from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import *
from pyspark.sql import functions as func
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [ ]:
# Download the data
! wget -q https://s3.amazonaws.com/metcs777/person_knows_person.csv
! wget -q https://s3.amazonaws.com/metcs777/person_likes_post.csv
! wget -q https://s3.amazonaws.com/metcs777/post_hasCreator_person.csv
! wget -q https://s3.amazonaws.com/metcs777/comment_hasCreator_person.csv
! wget -q https://s3.amazonaws.com/metcs777/comment_replyOf_post.csv


In [ ]:
! ls -la 

total 33572
drwxr-xr-x  1 root root     4096 Nov 12 00:50 .
drwxr-xr-x  1 root root     4096 Nov 12 00:41 ..
-rw-r--r--  1 root root  7398677 Dec 12  2018 comment_hasCreator_person.csv
-rw-r--r--  1 root root  3392677 Dec 12  2018 comment_replyOf_post.csv
drwxr-xr-x  1 root root     4096 Nov  6 17:30 .config
-rw-r--r--  1 root root   232246 Dec 12  2018 person_knows_person.csv
-rw-r--r--  1 root root 21372777 Aug 15 19:14 person_likes_post.csv
-rw-r--r--  1 root root  1950946 Dec 12  2018 post_hasCreator_person.csv
drwxr-xr-x  1 root root     4096 Nov  6 17:30 sample_data
drwxr-xr-x 13 1000 1000     4096 Aug 28 09:22 spark-3.0.1-bin-hadoop3.2


In [ ]:
!pwd

/content


In [ ]:
# Set your file path here 
path="file:///content/"

# You have 5 files 

# comment_hasCreator_person.csv
# comment_replyOf_post.csv
# person_knows_person.csv
# person_likes_post.csv
# post_hasCreator_person.csv

fileCommentHasCreator= path+"comment_hasCreator_person.csv"
fileComment_replyOf_post = path+"comment_replyOf_post.csv"
filePerson_knows_person= path+"person_knows_person.csv"
filePerson_likes_post = path + "person_likes_post.csv"
filePost_hasCreator_person = path + "post_hasCreator_person.csv"

#Create RDDs 

In [ ]:
def getRDD(mfile, sc):
		lines = sc.textFile(mfile)
		# Removing the Header Line 
		linesHeader = lines.first()
		header = sc.parallelize([linesHeader])
		linesWithOutHeader = lines.subtract(header)
		myRDD = linesWithOutHeader.map(lambda x: x.split('|'))
		return myRDD

In [ ]:
	# Create RDDs from files 
	commentHasCreator = getRDD(fileCommentHasCreator, sc)
	comment_replyOf_post = getRDD(fileComment_replyOf_post, sc)
	person_knows_person = getRDD(filePerson_knows_person, sc)
	person_likes_post = getRDD(filePerson_likes_post, sc)
	post_hasCreator_person = getRDD(filePost_hasCreator_person, sc)

In [ ]:
person_likes_post.take(10)

[['47', '20', '2012-10-23T03:32:14Z'],
 ['378', '20', '2012-10-25T01:03:21Z'],
 ['592', '20', '2012-10-18T06:02:36Z'],
 ['724', '20', '2012-10-23T00:17:38Z'],
 ['507', '30', '2012-06-23T07:15:14Z'],
 ['576', '40', '2012-12-31T05:50:25Z'],
 ['672', '40', '2012-12-27T02:02:20Z'],
 ['711', '40', '2012-12-31T10:22:23Z'],
 ['417', '50', '2012-09-05T00:59:45Z'],
 ['109', '50', '2012-09-04T11:24:35Z']]

In [ ]:
commentHasCreator.take(10)

[['30', '457'],
 ['40', '956'],
 ['90', '6'],
 ['120', '962'],
 ['170', '48'],
 ['230', '956'],
 ['260', '29'],
 ['320', '6'],
 ['380', '501'],
 ['440', '498']]

In [ ]:
commentHasCreator.join()

# Create Dataframes

In [ ]:
commentHasCreatorDD = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(fileCommentHasCreator)
comment_replyOf_postDD = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(fileComment_replyOf_post)
person_knows_personDD = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(filePerson_knows_person)
person_likes_postDD = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(filePerson_likes_post)
post_hasCreator_personDD = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(filePost_hasCreator_person)


In [ ]:
commentHasCreatorDD.show()
comment_replyOf_postDD.show()
person_knows_personDD.show()
person_likes_postDD.show()
post_hasCreator_personDD.show()


+----------+---------+
|Comment.id|Person.id|
+----------+---------+
|         0|       74|
|        10|      832|
|        20|      913|
|        30|      457|
|        40|      956|
|        50|       41|
|        60|      453|
|        70|      832|
|        80|        6|
|        90|        6|
|       100|      103|
|       110|      547|
|       120|      962|
|       130|       99|
|       140|      452|
|       150|       99|
|       160|        6|
|       170|       48|
|       180|      941|
|       190|       40|
+----------+---------+
only showing top 20 rows

+----------+-------+
|Comment.id|Post.id|
+----------+-------+
|         0|      0|
|        10|      0|
|        30|      0|
|        70|      0|
|       100|     10|
|       110|     10|
|       140|     10|
|       150|     10|
|       180|     10|
|       240|     20|
|       250|     20|
|       330|     20|
|       340|     30|
|       350|     30|
|       420|     30|
|       470|     40|
|       480|     40|
| 

In [ ]:
commentHasCreatorDD = commentHasCreatorDD.withColumnRenamed("Comment.id", "Comment").withColumnRenamed("Person.id", "Person")
comment_replyOf_postDD = comment_replyOf_postDD.withColumnRenamed("Comment.id", "Comment").withColumnRenamed("Post.id", "Post")
person_knows_personDD = person_knows_personDD.withColumnRenamed("Person.id0", "Id0").withColumnRenamed("Person.id1", "Id1")
person_likes_postDD = person_likes_postDD.withColumnRenamed("Person.id","Person").withColumnRenamed("Post.id","Post")
temp = commentHasCreatorDD.join(comment_replyOf_postDD, commentHasCreatorDD.Comment == comment_replyOf_postDD.Comment).drop(commentHasCreatorDD.Comment)
temp2  = temp.join(person_knows_personDD, temp.Person == person_knows_personDD.Id1).drop(person_knows_personDD.Id1)
temp3 = temp2.join(person_likes_postDD, temp2.Person == person_likes_postDD.Person).drop(person_likes_postDD.Person)
temp3.show()


+------+-------+-----+---+-----+-------------------+
|Person|Comment| Post|Id0| Post|       creationDate|
+------+-------+-----+---+-----+-------------------+
|   463|  54790|10780|458|10440|2012-08-28 02:39:27|
|   463|  54790|10780|458|10450|2012-10-28 10:50:42|
|   463|  54790|10780|458|10520|2012-09-10 07:26:36|
|   463|  54790|10780|458|10600|2012-07-15 11:47:32|
|   463|  54790|10780|458|10900|2012-09-18 10:13:46|
|   463|  54790|10780|458|11000|2012-07-26 04:46:03|
|   463|  54790|10780|458|11030|2012-10-08 02:50:38|
|   463|  54790|10780|458|11150|2012-08-26 07:01:04|
|   463|  54790|10780|458|11210|2012-08-21 10:47:06|
|   463|  54790|10780|458|11350|2013-01-20 02:39:15|
|   463|  54790|10780|458|11360|2012-12-27 02:41:59|
|   463|  54790|10780|458|11480|2012-11-27 08:11:34|
|   463|  54790|10780|458|11500|2012-10-11 08:46:01|
|   463|  54790|10780|458|11520|2012-06-04 04:19:34|
|   463|  54790|10780|458|11550|2012-09-04 00:50:34|
|   463|  54790|10780|458|11670|2012-12-29 04:

In [ ]:
from pyspark.sql import functions as F
temp3 = temp3.withColumn("num", lit(1))
tempAnswer = temp3.groupBy("Person").agg(F.sum('num').alias('sum'))
numActivity = tempAnswer.orderBy(desc("sum")).limit(10)
numActivity.take(10)